In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import minimize


Conisdere o modelo de regressão

\begin{equation}
y_t=X_t\beta +u_t, \quad u_t \sim N(0, \sigma^2)
\end{equation}



In [68]:
# Simulated data
np.random.seed(1234)

nobs = 1000

x1 = np.random.normal(size=nobs)
x2 = np.random.normal(size=nobs)
x3 = np.random.normal(size=nobs)

X = np.vstack([np.ones(nobs), x1, x2, x3]).T

sigma2 = 2
beta = np.array([5, -0.9, 0.5, 0.01])

y = X.dot(beta) + np.random.normal(size=nobs, scale=sigma2)

In [69]:
# OLS
b_ols = np.linalg.solve(X.T@X, X.T@y)
b_ols

array([ 4.96077633, -0.94240586,  0.53071216,  0.0275337 ])

In [70]:
def neg_log_like_unconst(params, y, X):
    nobs = len(y)
    beta = params[1:]
    
    log_like = 0
    for t in range(nobs):
        uhat = y[t] - X[t,:]@beta
        log_like += -0.5*(np.log(2*np.pi*params[0]) + (uhat**2)/params[0])
    
    return -log_like

In [71]:
res = minimize(fun=neg_log_like_unconst, 
               x0=np.array([np.var(y), np.mean(y), 0, 0, 0]),
               args=(y,X), method="Nelder-Mead")

In [72]:
b_mle = res.x[1:]
b_mle

array([ 4.96077669, -0.94240874,  0.53068865,  0.0275428 ])

In [73]:
def neg_log_like_const(params, y, X):
    nobs = len(y)
    beta1 = params[1]
    beta2 = params[2]
    beta3 = np.exp(params[3]) # constrain
    beta4 = params[4]
    
    beta = np.array([beta1, beta2, beta3, beta4])
    
    log_like = 0
    for t in range(nobs):
        uhat = y[t] - X[t,:]@beta
        log_like += -0.5*(np.log(2*np.pi*params[0]) + (uhat**2)/params[0])
    
    return -log_like

In [74]:
res = minimize(fun=neg_log_like_const, 
               x0=np.array([np.var(y), np.mean(y), 0, 0, 0]),
               args=(y,X), method="Nelder-Mead")

In [75]:
res.x[1:]

array([ 4.96077198, -0.9424218 , -0.63348733,  0.02752746])

In [76]:
np.exp(res.x[3])

0.5307377137236197

In [61]:
import pystan

In [62]:
model= '''data {
    int<lower=0> N;   // number of data items
    int<lower=0> K;   // number of predictors
    matrix[N, K] x;   // predictor matrix
    vector[N] y;      // outcome vector
}
parameters {
    vector[K] beta;       // coefficients for predictors
    real<lower=0> sigma;  // error scale
}
model {
    for (i in 1:K){
        if (i==3){
            beta[i]~normal(0,100); // restringe esse parâmetro para ser positivo
        }
        else{
            beta[i] ~ normal(0, 100); // normal with mean=0 and std=10
        }
    }
    
    y ~ normal(x * beta , sigma);  // likelihood
}'''

In [63]:

sm = pystan.StanModel(model_code=model) # compila o modelo

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_452bf1e73990ae3db0322706637bfddf NOW.


In [77]:
data = {"N": len(y),
        "K": X.shape[1],
        "y": y,
        "x": X}


fit = sm.sampling(data=data, iter=1000, chains=2, seed=1, verbose=True) # roda as iterações

In [78]:
b_bayes = np.mean(fit.extract()["beta"],axis=0)
b_bayes

array([ 4.9601393 , -0.94221204,  0.53256816,  0.02811377])

In [79]:
estimators=pd.DataFrame()

estimators["OLS"] = b_ols
estimators["MLE"] = b_mle
estimators["Bayes"]=b_bayes

In [80]:
estimators

,OLS,MLE,Bayes
0,4.960776,4.960777,4.960139
1,-0.942406,-0.942409,-0.942212
2,0.530712,0.530689,0.532568
3,0.027534,0.027543,0.028114
